In [2]:
!pip install kenlm streamlit pyngrok pyvi autocorrect nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=1cb0c832a3ac5144146ee6c081efa7b69a06a936d12dae9818abaa4e0fe2c5ae
  Stored in directory: /root/.cache/pip/wheels/5e/90/99/807a5ad861ce5d22c3c299a11df8cba9f31524f23ae6e645cb
Successfully built autocorrect


In [6]:
%%writefile app.py
import streamlit as st
import kenlm
from pyvi import ViTokenizer
import os
from autocorrect import Speller
import nltk
from nltk.metrics.distance import edit_distance
from nltk.corpus import words

nltk.download('words')

@st.cache_resource
def load_model():
    model_path = '/content/3-gram-lm.binary'
    if not os.path.exists(model_path):
      st.error("Model not found. Please upload the model file.")
      return None
    else:
      return kenlm.Model(model_path)

@st.cache_resource
def load_data():
  data_path = '/content/Viet74K.txt'
  if not os.path.exists(data_path):
    st.error("Data not found. Please upload the data file.")
    return []
  with open(data_path, 'r', encoding='utf-8') as f:
      return [line.strip() for line in f if line.strip()]

def correct_spelling(text, dictionary):
    """Hàm sửa lỗi chính tả cho văn bản tiếng Việt"""
    tokenized = ViTokenizer.tokenize(text).split()
    corrected = []

    for word in tokenized:
        if word in dictionary:
            corrected.append(word)
        else:
            # Tìm từ gần nhất
            distances = [(w, edit_distance(word, w)) for w in dictionary]
            distances.sort(key=lambda x: x[1])
            # Lấy từ có khoảng cách nhỏ nhất (nếu khoảng cách hợp lý, ví dụ < 3)
            if distances and distances[0][1] < 3:
                corrected.append(distances[0][0])
            else:
                corrected.append(word)  # Giữ nguyên nếu không tìm thấy từ gần đúng
    return " ".join(corrected)

def suggest_next_words(prefix, model, data, top_n=3):
    if not prefix.strip():
        return []
    tokenized_prefix = ViTokenizer.tokenize(prefix).strip()
    scores = []
    for word in data:
        sentence = tokenized_prefix + " " + word
        score = model.score(sentence)
        scores.append((word, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_n]

st.title("Vietnamese Autocomplete with Spell Correction")
st.write("Nhập một cụm từ, hệ thống sẽ tự động sửa lỗi chính tả và gợi ý từ tiếp theo sử dụng mô hình KenLM")

# Tải model và từ điển
model = load_model()
dictionary = load_data()
if not model or not dictionary:
    st.stop()

# Khởi tạo session state để lưu trữ input đã sửa
if 'corrected_input' not in st.session_state:
    st.session_state.corrected_input = ""

# Hàm callback khi input thay đổi
def on_input_change():
    user_input = st.session_state.input_text
    if user_input.strip():
        # Sửa lỗi chính tả
        corrected_text = correct_spelling(user_input, dictionary)
        st.session_state.corrected_input = corrected_text
    else:
        st.session_state.corrected_input = ""

# Text input với callback
input_text = st.text_input(
    "Nhập cụm từ (ví dụ: 'ăn cơm với'): ",
    placeholder="Nhập cụm từ...",
    key="input_text",
    on_change=on_input_change
)

# Hiển thị văn bản đã sửa
if st.session_state.corrected_input:
    st.write(f"Văn bản đã sửa: {st.session_state.corrected_input}")

top_n = st.slider("Số lượng từ gợi ý: ", min_value=1, max_value=5, value=3)

if st.button("Gợi ý"):
    if st.session_state.corrected_input.strip():
        with st.spinner("Đang xử lý..."):
            suggestions = suggest_next_words(st.session_state.corrected_input, model, dictionary, top_n)
            if suggestions:
                st.success("Kết quả gợi ý:")
                for i, (word, score) in enumerate(suggestions, 1):
                    st.write(f"{i}. {word} (score: {score:.2f})")
            else:
                st.warning("Không tìm thấy từ tiếp theo phù hợp")
    else:
        st.error("Vui lòng nhập một cụm từ!")

st.markdown("---")

Overwriting app.py


In [9]:
from pyngrok import ngrok

!ngrok authtoken 30nuioI87pSwDlc4HKht0NpkTVq_5zTmzA2tAfrae4JKWbGwU

public_url = ngrok.connect(8501)
print(f"Truy cập giao diện Streamlit tại: {public_url}")

# Chạy Streamlit
!streamlit run app.py

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Truy cập giao diện Streamlit tại: NgrokTunnel: "https://88868911aedc.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.48.254:8501

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words i